see: https://www.overleaf.com/read/knktnhbycdzg#210acd

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

L = np.pi
N = 256

theta = np.linspace(-np.pi, np.pi, N)

def plot_V_eta_and_delta_W(A, B, C, w0, w1):
    V = lambda theta: A + B * np.cos(theta) + C * np.cos(theta)**2
    
    V_non_negative = np.abs(V(theta)) #np.maximum(V(theta), 0)
    delta_W = lambda theta: np.sqrt(V_non_negative) * np.random.randn(len(theta))

    W = lambda delta_theta: w0 + w1 * np.cos(delta_theta) + delta_W(delta_theta)

    fig, axs = plt.subplots(1, 3, figsize=(24, 6)) 

    axs[0].plot(theta, V(theta), 'r-', markersize=2)
    axs[0].axhline(0, color='gray', linestyle='--')
    axs[0].set_xlabel('Theta')
    axs[0].set_ylabel('V(theta)')
    axs[0].set_title('V(theta)')
    axs[0].grid(True)
    axs[0].legend(['V'])

    delta_W_values = delta_W(theta)
    axs[1].plot(theta, delta_W_values, 'b-', markersize=2)
    axs[1].set_xlabel('Theta')
    axs[1].set_ylabel('delta_W(theta)')
    axs[1].set_title('delta_W(theta)')
    axs[1].grid(True)
    axs[1].legend(['delta_W'])

    W_values = W(theta)
    axs[2].plot(theta, W_values, 'g-', markersize=2)
    axs[2].set_xlabel('Theta')
    axs[2].set_ylabel('W(theta)')
    axs[2].set_title('W(theta)')
    axs[2].grid(True)
    axs[2].legend(['W'])

    plt.show()

A_slider = FloatSlider(min=-5, max=5, step=1, value=0.5, description='A')
B_slider = FloatSlider(min=-5, max=5, step=1, value=0, description='B')
C_slider = FloatSlider(min=-5, max=5, step=1, value=0, description='C')
w0_slider = FloatSlider(min=-30, max=30, step=1, value=-10, description='w0')
w1_slider = FloatSlider(min=-30, max=30, step=1, value=10, description='w1')

interact(plot_V_eta_and_delta_W, A=A_slider, B=B_slider, C=C_slider, w0=w0_slider, w1=w1_slider)


interactive(children=(FloatSlider(value=0.5, description='A', max=5.0, min=-5.0, step=1.0), FloatSlider(value=…

<function __main__.plot_V_eta_and_delta_W(A, B, C, w0, w1)>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, FloatSlider
import ipywidgets as widgets

def V(theta, A, B, C):
    return A + B * np.cos(theta) + C * np.cos(theta)**2

def generate_noise(N):
    return np.random.randn(N)

def delta_W(theta, A, B, C, noise):
    return np.sqrt(V(theta, A, B, C)) * noise

def fourier_coefficients(j, theta_values, delta_W_values):
    cos_terms = np.cos(j * theta_values)
    sin_terms = np.sin(j * theta_values)
    alpha_j = np.trapz(delta_W_values * cos_terms, theta_values) / (2 * np.pi)
    beta_j = np.trapz(delta_W_values * sin_terms, theta_values) / (2 * np.pi)
    return alpha_j, beta_j

def fourier_series(theta_values, J, delta_W_values):
    fourier_reconstruction = np.zeros_like(theta_values)
    contributions = np.zeros_like(theta_values)
    amplitude = 0  
    phase = 0      
    for j in range(1, J+1):
        alpha_j, beta_j = fourier_coefficients(j, theta_values, delta_W_values)
        contribution = alpha_j * np.cos(j * theta_values) + beta_j * np.sin(j * theta_values)
        fourier_reconstruction += contribution
        if j == J:
            contributions = contribution
            amplitude = np.sqrt(alpha_j**2 + beta_j**2)
            phase = np.arctan2(beta_j, alpha_j)
    return fourier_reconstruction, contributions, amplitude, phase

default_N = 64
noise = generate_noise(default_N)

def interactive_plot(J, A, B, C, N, w0, w1):
    theta_values = np.linspace(-np.pi, np.pi, N)
    global noise
    if len(noise) != N:
        noise = generate_noise(N)
    
    delta_W_values = delta_W(theta_values, A, B, C, noise)
    reconstructed_signal, last_mode_contribution, amplitude, phase = fourier_series(theta_values, J, delta_W_values)

    R_mean = np.sqrt(2 * np.pi / N * (A + C / 2)) # * sqrt(S / kappa)
    print(f"⟨R⟩ = {R_mean:.3f}")
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 6))
    
    ax1.plot(theta_values, delta_W_values, label='Original $\Delta W$', alpha=0.5)
    ax1.plot(theta_values, reconstructed_signal, label=f'Fourier Series up to {J} modes', linestyle='--')
    ax1.set_title("Fourier Series Approximation of $\Delta W$")
    ax1.set_xlabel("$\\theta$")
    ax1.set_ylabel("$\Delta W$")
    ax1.legend()
    
    ax2.plot(theta_values, last_mode_contribution, color='red',
             label=f'Contribution of Mode {J}\nAmplitude: {amplitude:.3f}\nPhase: {phase:.3f} rad')
    ax2.set_title(f"Contribution of Mode {J}")
    ax2.set_xlabel("$\\theta$")
    ax2.set_ylabel("Contribution to $\Delta W$")
    ax2.legend()

    kernel = w0 + w1 * np.cos(theta_values) + reconstructed_signal
    ax3.plot(theta_values, kernel, color='blue', label='Kernel $W(\\delta \\theta)$')
    ax3.set_title(f"Kernel with Fourier Series truncated at {J}")
    ax3.set_xlabel("$\\delta \\theta$")
    ax3.set_ylabel("$W(\\delta \\theta)$")
    ax3.legend()

interactive_plot = interactive(
    interactive_plot,
    J=IntSlider(min=0, max=128, step=1, value=0, description='J: Number of Modes'),
    A=FloatSlider(min=0, max=9, step=1, value=1, description='A'),
    B=FloatSlider(min=0, max=9, step=1, value=0, description='B'),
    C=FloatSlider(min=0, max=9, step=1, value=0, description='C'),
    N=IntSlider(min=16, max=128, step=16, value=default_N, description='N: Number of Points'),
    w0=FloatSlider(min=-20, max=10, step=1, value=-10, description='w0'),
    w1=FloatSlider(min=-20, max=20, step=1, value=10, description='w1')
)
display(interactive_plot)


interactive(children=(IntSlider(value=0, description='J: Number of Modes', max=128), FloatSlider(value=1.0, de…

In [3]:
from common_utils import Ring, derivative_nonlinearity, r_02
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

np.random.seed(42)

L = np.pi
N = 255
T = {'t_span': (0, 10000), 't_steps': 10000}

w0 = -10
I_0 = 0.9

A = 0.05
B = 0.0 
C = 0.0

M = 60  
s = 0.1  
kappa = s*M

R = np.sqrt((2*np.pi/N)*(A+(C/2))) * (np.sqrt(s*M)/kappa)

r_0 = r_02(w0, I_0)
critical_w1 = 2 / derivative_nonlinearity(w0 * r_0 + I_0) -  R
print(f'Critical W_1: {critical_w1:.4f}')
print(f'Previous Critical W_1 at Delta: {R:.4f}')

print(f'Solution r0={r_0:.4f}')

def update_plots(epsilon, delta):
    theta = np.linspace(-np.pi, np.pi, N)
    w1 = critical_w1 + delta
    perturbation = lambda theta: r_0 + epsilon * np.cos(theta)

    V = lambda theta: A + B * np.cos(theta) + C * np.cos(theta)**2
    
    delta_W = lambda theta: np.sqrt(V(theta)) * np.random.randn()
    
    W = lambda delta_theta: w0 + w1 * np.cos(delta_theta) + delta_W(delta_theta)

    ring = Ring(L, T, N, W, I_0, perturbation)

    fig, axs = plt.subplots(1, 3, figsize=(24, 6))  

    ring.plot_timetrace(ax=axs[0], phase=0)
    ring.plot_state(ax=axs[1], timestep=-1)
    ring.plot_dynamics(ax=axs[2])
    
    plt.tight_layout()
    plt.show()

epsilon_slider = FloatSlider(value=0.005, min=0.0, max=0.1, step=0.005, description='Epsilon:', readout_format='.3f')
delta_slider = FloatSlider(value=0.0, min=-0.5, max=0.5, step=0.05, description='Delta:', readout_format='.2f')

interact(update_plots, epsilon=epsilon_slider, delta=delta_slider)

Critical W_1: 3.9205
Previous Critical W_1 at Delta: 0.0143
Solution r0=0.0646


interactive(children=(FloatSlider(value=0.005, description='Epsilon:', max=0.1, readout_format='.3f', step=0.0…

<function __main__.update_plots(epsilon, delta)>